# Building the NER for Nepali

### Dataset Preparation

We can train our own model for NER using spacy

For this we need a notated data in spacy format.

There are tools that can generate that particular formats:

- https://tecoholic.github.io/ner-annotator/

In [27]:
%pip install spacy nltk scikit-learn

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 10.6 MB/s  0:00:00 10.8 MB/s eta 0:00:01
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 11.4 MB/s  0:00:01 11.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [nltk]━━━━━━ 5/6 [nltk] [scikit-learn]
Note: you may need to restart the kernel to use updated packages.


In [28]:
#load annotated data

import json
with open('spacy-ner/train-ner-annotations.json', 'r') as f:
    data = json.load(f)

In [29]:
data

{'classes': ['PERSON', 'PLACE', 'ORGANIZATION', 'POST'],
 'annotations': [['संस्कृति, पर्यटन तथा नागरिक उड्डयनमन्त्री बद्री पाण्डेले विमान दुर्घटनाबारे सरकार गम्भीर रहेको बताएका छन् । गम्भीर रूपमा मन्त्रालय अध्ययनमा लागेको र चाँडै कारबाही लगायतका निर्णय लिइने उनले संसदीय समितिलाई जानकारी दिएका छन् । ',
   {'entities': [[28, 41, 'POST'],
     [42, 56, 'PERSON'],
     [189, 204, 'ORGANIZATION']]}],
  ['प्रतिनिधिसभा अन्तर्गतको अन्तर्राष्ट्रिय सम्बन्ध र पर्यटन समितिको बिहीबारको बैठकमा सांसदहरूले विमान र हेलिकप्टर दुर्घनाबारे सोधेका प्रश्नको जवाफ दिने क्रममा मन्त्री पाण्डेले कानुन र संवैधानिक रूपमा परिपक्व निर्णय लिन लागिएको बताएका हुन् ।',
   {'entities': [[0, 23, 'ORGANIZATION'],
     [24, 64, 'ORGANIZATION'],
     [156, 163, 'POST'],
     [164, 172, 'PERSON']]}],
  ["'मातहतका निकायलाई कारबाही गर्ने सम्बन्धमा घोत्लिएका छौं । खासगरी सौर्य एयरलाइन्सको विमान दुर्घटनापछि हामी घोत्लिएका छौं,' मन्त्री पाण्डेले भने, 'म के चाहन्न भने मन्त्रीको पहलकदमीमा कुनै निर्णय होस् र अर्को कुनै बाटोबाट पुनर्

In [30]:
data['classes']

['PERSON', 'PLACE', 'ORGANIZATION', 'POST']

In [31]:
data['annotations'][0]

['संस्कृति, पर्यटन तथा नागरिक उड्डयनमन्त्री बद्री पाण्डेले विमान दुर्घटनाबारे सरकार गम्भीर रहेको बताएका छन् । गम्भीर रूपमा मन्त्रालय अध्ययनमा लागेको र चाँडै कारबाही लगायतका निर्णय लिइने उनले संसदीय समितिलाई जानकारी दिएका छन् । ',
 {'entities': [[28, 41, 'POST'],
   [42, 56, 'PERSON'],
   [189, 204, 'ORGANIZATION']]}]

In [32]:
data['annotations'][0][1]

{'entities': [[28, 41, 'POST'],
  [42, 56, 'PERSON'],
  [189, 204, 'ORGANIZATION']]}

In [33]:
print(data['annotations'][0][1]['entities'])

for entity in data['annotations'][0][1]['entities']:
    print(entity)

[[28, 41, 'POST'], [42, 56, 'PERSON'], [189, 204, 'ORGANIZATION']]
[28, 41, 'POST']
[42, 56, 'PERSON']
[189, 204, 'ORGANIZATION']


Exploring what those numbers are

In [34]:
full_text = data['annotations'][0][0]
full_text

'संस्कृति, पर्यटन तथा नागरिक उड्डयनमन्त्री बद्री पाण्डेले विमान दुर्घटनाबारे सरकार गम्भीर रहेको बताएका छन् । गम्भीर रूपमा मन्त्रालय अध्ययनमा लागेको र चाँडै कारबाही लगायतका निर्णय लिइने उनले संसदीय समितिलाई जानकारी दिएका छन् । '

In [35]:
print(full_text[28:41])
print(full_text[42: 56])

उड्डयनमन्त्री
बद्री पाण्डेले


### Translate and Load data

In [19]:
import json

In [36]:
# load data
with open('spacy-ner/train-ner-annotations.json', 'r') as f:
    data = json.load(f)
data


{'classes': ['PERSON', 'PLACE', 'ORGANIZATION', 'POST'],
 'annotations': [['संस्कृति, पर्यटन तथा नागरिक उड्डयनमन्त्री बद्री पाण्डेले विमान दुर्घटनाबारे सरकार गम्भीर रहेको बताएका छन् । गम्भीर रूपमा मन्त्रालय अध्ययनमा लागेको र चाँडै कारबाही लगायतका निर्णय लिइने उनले संसदीय समितिलाई जानकारी दिएका छन् । ',
   {'entities': [[28, 41, 'POST'],
     [42, 56, 'PERSON'],
     [189, 204, 'ORGANIZATION']]}],
  ['प्रतिनिधिसभा अन्तर्गतको अन्तर्राष्ट्रिय सम्बन्ध र पर्यटन समितिको बिहीबारको बैठकमा सांसदहरूले विमान र हेलिकप्टर दुर्घनाबारे सोधेका प्रश्नको जवाफ दिने क्रममा मन्त्री पाण्डेले कानुन र संवैधानिक रूपमा परिपक्व निर्णय लिन लागिएको बताएका हुन् ।',
   {'entities': [[0, 23, 'ORGANIZATION'],
     [24, 64, 'ORGANIZATION'],
     [156, 163, 'POST'],
     [164, 172, 'PERSON']]}],
  ["'मातहतका निकायलाई कारबाही गर्ने सम्बन्धमा घोत्लिएका छौं । खासगरी सौर्य एयरलाइन्सको विमान दुर्घटनापछि हामी घोत्लिएका छौं,' मन्त्री पाण्डेले भने, 'म के चाहन्न भने मन्त्रीको पहलकदमीमा कुनै निर्णय होस् र अर्को कुनै बाटोबाट पुनर्

In [37]:
training_dataset = [ ]

# prepare the data from the json file
for annotation in data['annotations']:
    # check for the data
    # print(annotation[0])
    # print(annotation[1])
    if annotation is None: continue
    temp_data = { }
    temp_data['text'] = annotation[0]
    temp_data['entities'] = [ ]

    for entity in annotation[1]['entities']:
        # print(entity)
        start, end, label = entity
        # print(start, end, label)
        temp_data['entities'].append((start, end, label))
    training_dataset.append(temp_data)

In [38]:
# dataset
training_dataset

[{'text': 'संस्कृति, पर्यटन तथा नागरिक उड्डयनमन्त्री बद्री पाण्डेले विमान दुर्घटनाबारे सरकार गम्भीर रहेको बताएका छन् । गम्भीर रूपमा मन्त्रालय अध्ययनमा लागेको र चाँडै कारबाही लगायतका निर्णय लिइने उनले संसदीय समितिलाई जानकारी दिएका छन् । ',
  'entities': [(28, 41, 'POST'),
   (42, 56, 'PERSON'),
   (189, 204, 'ORGANIZATION')]},
 {'text': 'प्रतिनिधिसभा अन्तर्गतको अन्तर्राष्ट्रिय सम्बन्ध र पर्यटन समितिको बिहीबारको बैठकमा सांसदहरूले विमान र हेलिकप्टर दुर्घनाबारे सोधेका प्रश्नको जवाफ दिने क्रममा मन्त्री पाण्डेले कानुन र संवैधानिक रूपमा परिपक्व निर्णय लिन लागिएको बताएका हुन् ।',
  'entities': [(0, 23, 'ORGANIZATION'),
   (24, 64, 'ORGANIZATION'),
   (156, 163, 'POST'),
   (164, 172, 'PERSON')]},
 {'text': "'मातहतका निकायलाई कारबाही गर्ने सम्बन्धमा घोत्लिएका छौं । खासगरी सौर्य एयरलाइन्सको विमान दुर्घटनापछि हामी घोत्लिएका छौं,' मन्त्री पाण्डेले भने, 'म के चाहन्न भने मन्त्रीको पहलकदमीमा कुनै निर्णय होस् र अर्को कुनै बाटोबाट पुनर्स्थापति होस् र मन्त्री त्यहीँ बसिरहोस्, मेरो निर्णयमा चुनौती जुनसुक

In [39]:
training_dataset[0]['text']

'संस्कृति, पर्यटन तथा नागरिक उड्डयनमन्त्री बद्री पाण्डेले विमान दुर्घटनाबारे सरकार गम्भीर रहेको बताएका छन् । गम्भीर रूपमा मन्त्रालय अध्ययनमा लागेको र चाँडै कारबाही लगायतका निर्णय लिइने उनले संसदीय समितिलाई जानकारी दिएका छन् । '

In [40]:
training_dataset[0]['entities']

[(28, 41, 'POST'), (42, 56, 'PERSON'), (189, 204, 'ORGANIZATION')]

### Spacy compatible dataset

- **Conversion to spaCy Format:** It transforms annotated data into a format compatible with spaCy v3, creating Doc objects with character spans linked to entities.

- **Entity Alignment:** Prevents entity overlaps or conflicts within documents, avoiding training issues.

In [41]:
import spacy
from spacy.tokens import DocBin
from spacy.util import filter_spans
from tqdm import tqdm

In [42]:
def build_docbin(dataset_):
    nlp = spacy.blank("ne") # load a new spacy model
    doc_bin = DocBin()
    # iterate over data
    for training_data  in tqdm(dataset_):
        text = training_data['text']
        labels = training_data['entities']
        doc = nlp.make_doc(text) 
        ents = []
        for start, end, label in labels:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        filtered_ents = filter_spans(ents)
        doc.ents = filtered_ents 
        doc_bin.add(doc)
    return doc_bin

In [43]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(training_dataset, test_size=0.2)

In [44]:
doc_bin_train = build_docbin(train)
doc_bin_train.to_disk("spacy-ner/train_data.spacy")

100%|██████████| 28/28 [00:00<00:00, 838.30it/s]


In [45]:
doc_bin_test = build_docbin(test)
doc_bin_test.to_disk("spacy-ner/test_data.spacy")

100%|██████████| 8/8 [00:00<00:00, 677.57it/s]


### Configuring SpaCy
This step involves configuring SpaCy for your custom NER model involves initializing configuration files, and you can use a base configuration file as a template. To download the base_config.cfg file from the documentation, you can visit the official spaCy do cumentation website: https://spacy.io/usage/training#quickstart 

```
# This is an auto-generated partial config. To use it with 'spacy train'
# you can run spacy init fill-config to auto-fill all default settings:
# python -m spacy init fill-config ./base_config.cfg ./config.cfg
[paths]
train = null
dev = null
vectors = null
[system]
gpu_allocator = null

[nlp]
lang = "en"
pipeline = ["tok2vec","ner"]
batch_size = 1000

[components]

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${components.tok2vec.model.encode.width}
attrs = ["NORM", "PREFIX", "SUFFIX", "SHAPE"]
rows = [5000, 1000, 2500, 2500]
include_static_vectors = false

[components.tok2vec.model.encode]
@architectures = "spacy.MaxoutWindowEncoder.v2"
width = 96
depth = 4
window_size = 1
maxout_pieces = 3

[components.ner]
factory = "ner"

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}

[corpora]

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${paths.train}
max_length = 0

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
max_length = 0

[training]
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"

[training.optimizer]
@optimizers = "Adam.v1"

[training.batcher]
@batchers = "spacy.batch_by_words.v1"
discard_oversize = false
tolerance = 0.2

[training.batcher.size]
@schedules = "compounding.v1"
start = 100
stop = 1000
compound = 1.001

[initialize]
vectors = ${paths.vectors}
```

After the base config we have to generate the full configuration file using `init-fill-config` 

In [46]:
!python -m spacy init fill-config spacy-ner/base_config.cfg spacy-ner/config.cfg

✔ Auto-filled config with all values
✔ Saved config
spacy-ner/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Train the model as per config file

In [47]:
!python -m spacy train spacy-ner/config.cfg  --output spacy-ner/trained_models  --paths.train spacy-ner/train_data.spacy  --paths.dev spacy-ner/test_data.spacy

ℹ Saving to output directory: spacy-ner/trained_models
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     26.44    0.00    0.00    0.00    0.00
 12     200        799.77   2027.82   48.98   54.55   44.44    0.49
 28     400       3981.22    152.19   57.14   63.64   51.85    0.57
 48     600         42.06     24.65   62.22   77.78   51.85    0.62
 73     800         23.79      6.42   59.09   76.47   48.15    0.59
105    1000        113.77     30.66   60.87   73.68   51.85    0.61
143    1200        177.65     15.50   45.83   52.38   40.74    0.46
192    1400        176.72     22.97   43.48   52.63   37.04    0.43
252    1600  

### Inference

In [48]:
import spacy
nlp = spacy.load('spacy-ner/trained_models/model-best')
doc =  nlp('दुर्घटनालगत्तै उद्धार गरेर क्याप्टेन शाक्यलाई काठमाण्डु विमानस्थलबाट नजिकै रहेको नेपाल अस्पताल पुर्‍याइएको थियो ।')

for ent in doc.ents:
    print(ent.text, " => ", ent.label_)

क्याप्टेन  =>  POST
शाक्यलाई  =>  PERSON
नेपाल अस्पताल  =>  ORGANIZATION
